<a href="https://colab.research.google.com/github/Blane187/Project/blob/main/songstarter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


#@title install

import subprocess
import os
subprocess.run(['pip', 'install', 'git+https://github.com/facebookresearch/audiocraft'])
subprocess.run(['pip', 'install', 'torchvision==0.16.0'])
subprocess.run(['pip', 'install', 'hf-transfer'])

print("done")

In [ ]:

#@markdown enables faster downloads from hugging face
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

In [ ]:
import torchaudio
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write
from huggingface_hub import hf_hub_download

model = MusicGen.get_pretrained('nateraw/musicgen-songstarter-v0.2')

 Generate music from text descriptions

In [ ]:


descriptions = "['acoustic, guitar, melody, trap, d minor, 90 bpm']" #@param {type:"string"}


model.set_generation_params(duration=10)  # generate 10 seconds.
descriptions =descriptions  * 3
wav = model.generate(descriptions)  # generates 3 samples.

for idx, one_wav in enumerate(wav):
    # Will save under {idx}.wav, with loudness normalization at -14 db LUFS.
    audio_write(f'{idx}', one_wav.cpu(), model.sample_rate, strategy="loudness", loudness_compressor=True)

In [ ]:
from IPython.display import Audio

# Listen to the first sample
Audio('0.wav')

We trained the model for text-to-music, but since the base model had melody conditioning, we get that for free.

That means you can generate samples that sound like some other sample you have.

Let's first listen to our melody sample:

In [ ]:
melody_path = hf_hub_download("nateraw/musicgen-songstarter-v0.2", 'assets/bach.mp3')
Audio(melody_path)

Now lets take that bach organ sample and remix it into a hip hop guitar sample

In [ ]:
melody, sr = torchaudio.load(melody_path)
# generates using the melody from the given audio and the provided descriptions.
wav = model.generate_with_chroma(descriptions, melody[None].expand(3, -1, -1), sr)

for idx, one_wav in enumerate(wav):
    # Will save under {idx}.wav, with loudness normalization at -14 db LUFS.
    audio_write(f'{idx}_bach', one_wav.cpu(), model.sample_rate, strategy="loudness", loudness_compressor=True)

In [ ]:
# Listen to the first melody conditioned sample
Audio('0_bach.wav')